In [ ]:
# Converting the given dataset to tensor format
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split
import numpy as np
import copy

transform = transforms.Compose([
    transforms.Resize((128,128)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.1),
    transforms.ToTensor()
])

data = datasets.ImageFolder("/kaggle/input/animals10/raw-img", transform = transform)
img, lab = data[0]
print(img.shape, lab)

In [ ]:
# splitting the given data into train, validation and test sets
print(len(data))
train_size = int(0.7 * len(data))
val_size = int(0.15 * len(data))
test_size = len(data) - train_size - val_size

train_data, val_data, test_data = random_split(data, [train_size, val_size, test_size]) 
print(train_size)

In [ ]:
data.classes

In [ ]:
# displaying our data
import matplotlib.pyplot as plt

def pic(img, lab):
    print("Label: ", translate[data.classes[lab]], "(", str(lab), ")")
    plt.imshow(img.permute(1,2,0))

In [ ]:
pic(*data[0])

In [ ]:
# Loading our data into batches

from torch.utils.data import DataLoader
train_dl = DataLoader(train_data, 100, shuffle = True)
val_dl = DataLoader(val_data, 100, shuffle = True)

In [ ]:
# Displaying a batch

from torchvision.utils import make_grid

def batch(dl):
    for img, lab in dl:
        fig, ax = plt.subplots(figsize = (10,10))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(img, 10).permute(1,2,0))
        break

In [ ]:
batch(train_dl)

In [ ]:
# Model Creation
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2),  # 128 → 64

    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2),  # 64 → 32

    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2),  # 32 → 16

    nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2),  # 16 → 8

    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(128*8*8, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 10)
)

In [ ]:
# checking our model

for img, lab in train_dl:
    out = model(img)
    print('out shape:', out.shape)
    print('out[0]', out[0])
    break

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(datas, device):
    if isinstance(datas, (list, tuple)):
        return [to_device(x, device) for x in datas]
    return datas.to(device, non_blocking = True)


class DeviceLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.device)


In [ ]:
# Checking the default device
device = get_default_device()
device

In [ ]:
#loading to device
train_dl = DeviceLoader(train_dl, device)
val_dl = DeviceLoader(val_dl, device)
to_device(model, device)

In [ ]:
def loss_batch(model, loss_fn, xb, yb, opt = None, metric = None):
    preds = model(xb)
    loss = loss_fn(preds, yb)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    metric_result = None
    if metric is not None:
        metric_result = metric(preds, yb)
    return loss.item(), len(xb), metric_result

In [ ]:

def eval_batch(model, loss_fn, valid_dl, metric = None):
    with torch.no_grad():
        results = [loss_batch(model, loss_fn, xb, yb, opt = None, metric = metric) for xb, yb in valid_dl]
        losses, nums, metrics = zip(*results)
# Convert to CPU floats
        losses = [l.item() if torch.is_tensor(l) else l for l in losses]
        nums = [n.item() if torch.is_tensor(n) else n for n in nums]
        if metric is not None:
            metrics = [m.item() if torch.is_tensor(m) else m for m in metrics]
        
        total = np.sum(nums)
        avg_loss = np.sum(np.multiply(losses, nums)) /  total
        avg_metric = None
        if metric is not None:
            avg_metric = np.sum(np.multiply(metrics, nums)) / total

    return avg_loss, total, avg_metric

In [ ]:
def fit(epochs, model, loss_fn, train_dl, valid_dl, opt_fn = None, lr = None, metric = None,
        weight_decay = 0.0001, patience=3):
    train_losses, val_losses, val_metrics = [], [], []
    if opt_fn is None:
        opt_fn = torch.optim.SGD
    opt = opt_fn(model.parameters(), lr = lr, weight_decay = weight_decay)
    
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            train_loss, _, _ = loss_batch(model, loss_fn, xb, yb, opt)

        model.eval()
        val_loss, total, val_metric = eval_batch(model, loss_fn, valid_dl, metric)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_metrics.append(val_metric)

        if metric is None:
            print('Epoch [{} / {}], train_loss: {:.4f}, val_loss: {:.4f}'
                  .format(epoch + 1, epochs, train_loss, val_loss))
        else: 
            print('Epoch [{} / {}], train_loss: {:.4f}, val_loss: {:.4f}, val_{} : {:.4f}'
                  .format(epoch + 1, epochs, train_loss, val_loss, metric.__name__, val_metric))


        # Early stopping logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_weights = copy.deepcopy(model.state_dict())
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                model.load_state_dict(best_model_weights)
                break
    return train_losses, val_losses, val_metrics

In [ ]:
# Creating a loss function and accuracy metric

import torch.nn.functional as F
loss_fn = nn.CrossEntropyLoss()


def acc(preds, y):
    predicted_classes = preds.argmax(dim=1)
    correct = (predicted_classes == y).sum()
    return correct.float() / y.size(0)

In [ ]:
# Storing the results for plotting
epochs = 20
opt_fn = torch.optim.Adam
lr = 0.001
metric = acc


history = fit(epochs, model, loss_fn, train_dl, val_dl, opt_fn, lr, metric, weight_decay = 0.0001, patience = 3)
train_losses, val_losses, val_metrics = history

In [ ]:
def plot_metric(metric_vals):
    plt.plot(metric_vals, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accurracy vs No. of Epochs')
    plt.show()

In [ ]:
plot_metric(val_metrics)

In [ ]:
def plot_losses(train, val):
    plt.plot(train, '-x', label='train')
    plt.plot(val, '-x', label='val')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss vs No. of Epochs')
    plt.legend()
    plt.show()

In [ ]:
plot_losses(train_losses, val_losses)